In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

# ---------- 設定 ----------
STOCK_CODE = "1101"
YEAR = "2024"  # 民國113年
MONTH = "4"    # 4月
OUTPUT_CSV = "twse_1101_202404.csv"
# ---------------------------

# 初始化 Selenium
options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # 若需背景執行可開啟
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(options=options)

try:
    driver.get("https://www.twse.com.tw/zh/trading/historical/stock-day-avg.html")

    wait = WebDriverWait(driver, 15)

    # 選擇年與月
    Select(wait.until(EC.presence_of_element_located((By.ID, "label0")))).select_by_value(YEAR)
    Select(wait.until(EC.presence_of_element_located((By.NAME, "mm")))).select_by_value(MONTH)

    # 輸入股票代號
    stock_input = wait.until(EC.presence_of_element_located((By.ID, "label1")))
    stock_input.clear()
    stock_input.send_keys(STOCK_CODE)

    # 點擊查詢按鈕（用 XPath + JS 點擊，最穩定）
    search_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "查詢")]')))
    driver.execute_script("arguments[0].click();", search_button)

    # 等待表格載入
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".is-last-page")))

    # 擷取表格內容
    rows = driver.find_elements(By.CSS_SELECTOR, ".is-last-page tr")
    data = []
    for row in rows:
        cols = [col.text.strip() for col in row.find_elements(By.TAG_NAME, "td")]
        if cols:
            data.append(cols)

    # 存入 pandas
    df = pd.DataFrame(data, columns=["日期", "收盤價"])
    print(df)

    # 存成 CSV（可選）
    df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
    print(f"✅ 資料已存成 {OUTPUT_CSV}")

except Exception as e:
    print("❌ 發生錯誤：", e)

finally:
    driver.quit()


           日期    收盤價
0   113/04/01  32.15
1   113/04/02  31.95
2   113/04/03  32.00
3   113/04/08  32.30
4   113/04/09  32.45
5   113/04/10  32.70
6   113/04/11  32.40
7   113/04/12  32.20
8   113/04/15  32.35
9   113/04/16  32.45
10  113/04/17  32.65
11  113/04/18  32.70
12  113/04/19  32.00
13  113/04/22  32.60
14  113/04/23  32.35
15  113/04/24  32.15
16  113/04/25  32.00
17  113/04/26  31.80
18  113/04/29  32.35
19  113/04/30  32.05
20     月平均收盤價  32.28
✅ 資料已存成 twse_1101_202404.csv


In [1]:
pip install webdriver


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement webdriver (from versions: none)
ERROR: No matching distribution found for webdriver


In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

# 讀取股票代號
with open("stock_ids.txt", encoding='utf-8') as f:
    stock_ids = sorted(set(line.strip() for line in f if line.strip().isdigit()))

# 設定年份與月份
years = [str(y) for y in range(2017, 2026)]  # 民國106年 ~ 114年
months = [str(m) for m in range(1, 13)]      # 1 ~ 12 月

# Chrome Driver 設定
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')

# 建立輸出資料夾
os.makedirs("data", exist_ok=True)

# 開始 Selenium 爬取
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 15)

try:
    for stock_id in stock_ids:
        for year in years:
            for month in months:
                filename = f"data/{stock_id}_{year}_{month.zfill(2)}.csv"
                if os.path.exists(filename):
                    continue  # 如果已存在就略過

                print(f"抓取 {stock_id} 年：{year} 月：{month}")
                try:
                    driver.get("https://www.twse.com.tw/zh/trading/historical/stock-day-avg.html")

                    # 年月股票代號
                    Select(wait.until(EC.presence_of_element_located((By.ID, "label0")))).select_by_value(year)
                    Select(driver.find_element(By.NAME, "mm")).select_by_value(month)
                    stock_input = driver.find_element(By.ID, "label1")
                    stock_input.clear()
                    stock_input.send_keys(stock_id)

                    # 查詢按鈕
                    search_btn = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "查詢")]')))
                    driver.execute_script("arguments[0].click();", search_btn)

                    # 等資料表格載入
                    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".is-last-page")))
                    rows = driver.find_elements(By.CSS_SELECTOR, ".is-last-page tr")
                    data = []
                    for row in rows:
                        cols = [col.text.strip() for col in row.find_elements(By.TAG_NAME, "td")]
                        if cols:
                            data.append(cols)

                    if data:
                        df = pd.DataFrame(data, columns=["日期", "收盤價"])
                        df.to_csv(filename, index=False, encoding="utf-8-sig")
                        print(f"✅ 已儲存：{filename}")
                    else:
                        print(f"⚠️ 無資料：{stock_id} {year}/{month}")

                except Exception as e:
                    print(f"❌ 錯誤：{stock_id} {year}/{month} => {e}")
                    continue  # 發生錯誤繼續下一個
finally:
    driver.quit()


抓取 1101 年：2017 月：1
✅ 已儲存：data/1101_2017_01.csv
抓取 1101 年：2017 月：2
✅ 已儲存：data/1101_2017_02.csv
抓取 1101 年：2017 月：3
✅ 已儲存：data/1101_2017_03.csv
抓取 1101 年：2017 月：4
✅ 已儲存：data/1101_2017_04.csv
抓取 1101 年：2017 月：5
✅ 已儲存：data/1101_2017_05.csv
抓取 1101 年：2017 月：6
✅ 已儲存：data/1101_2017_06.csv
抓取 1101 年：2017 月：7
✅ 已儲存：data/1101_2017_07.csv
抓取 1101 年：2017 月：8
✅ 已儲存：data/1101_2017_08.csv
抓取 1101 年：2017 月：9
✅ 已儲存：data/1101_2017_09.csv
抓取 1101 年：2017 月：10
✅ 已儲存：data/1101_2017_10.csv
抓取 1101 年：2017 月：11
❌ 錯誤：1101 2017/11 => Alert Text: 請稍候再試。
Message: unexpected alert open: {Alert text : 請稍候再試。}
  (Session info: chrome=135.0.7049.85)
Stacktrace:
	GetHandleVerifier [0x00007FF7A5B65335+78597]
	GetHandleVerifier [0x00007FF7A5B65390+78688]
	(No symbol) [0x00007FF7A59191AA]
	(No symbol) [0x00007FF7A59BFDFB]
	(No symbol) [0x00007FF7A5996EC3]
	(No symbol) [0x00007FF7A59603F8]
	(No symbol) [0x00007FF7A5961163]
	GetHandleVerifier [0x00007FF7A5E0EEED+2870973]
	GetHandleVerifier [0x00007FF7A5E09698+2848360]
	GetHa

KeyboardInterrupt: 

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# 設定
stock_code = "1101"
start_year = 2017
end_year = 2025
output_csv = f"{stock_code}_history.csv"

# 初始化瀏覽器
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # 除錯時可以先註解這行
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 15)

# 資料儲存陣列
all_data = []

try:
    for year in range(start_year, end_year + 1):
        driver.get("https://www.twse.com.tw/zh/trading/historical/fmsrfk.html")

        # 等待年份下拉選單出現
        wait.until(EC.presence_of_element_located((By.ID, "label0")))

        # 選擇民國年份（西元 = 民國 + 1911）
        select = Select(driver.find_element(By.ID, "label0"))
        select.select_by_value(str(year))

        # 輸入股票代碼
        stock_input = driver.find_element(By.ID, "label1")
        stock_input.clear()
        stock_input.send_keys(stock_code)

        # 查詢按鈕改用你給的 XPATH 方式來點擊
        search_btn = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "查詢")]')))
        driver.execute_script("arguments[0].scrollIntoView(true);", search_btn)
        time.sleep(0.3)
        driver.execute_script("arguments[0].click();", search_btn)

        # 等待結果表格載入
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "tbody.is-last-page")))

        # 擷取表格資料
        rows = driver.find_elements(By.CSS_SELECTOR, "tbody.is-last-page tr")
        for row in rows:
            cols = [cell.text.replace(",", "") for cell in row.find_elements(By.TAG_NAME, "td")]
            all_data.append(cols)

        print(f"✅ 已完成 民國 {year - 1911} 年資料")
        time.sleep(5)

finally:
    driver.quit()

# 輸出成 CSV
columns = ["年", "月", "最高", "最低", "收盤", "成交量(千股)", "成交金額", "市值", "報酬率%"]
df = pd.DataFrame(all_data, columns=columns)
df.to_csv(output_csv, index=False, encoding="utf-8-sig")
print(f"📁 CSV 檔已儲存：{output_csv}")



✅ 已完成 民國 106 年資料
✅ 已完成 民國 107 年資料


KeyboardInterrupt: 

In [19]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 設定
start_year = 2017
end_year = 2025
wait_time = 3  # 每次查詢間隔秒數
output_dir = "stock_data"

# 建立輸出資料夾
os.makedirs(output_dir, exist_ok=True)

# 讀取股票代碼
with open("stock_ids.txt", "r", encoding="utf-8") as f:
    stock_ids = sorted(set(line.strip() for line in f if line.strip()))

# 初始化瀏覽器
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 15)

# 查詢資料函數
def fetch_stock_data(stock_code):
    all_data = []

    for year in range(start_year, end_year + 1):
        driver.get("https://www.twse.com.tw/zh/trading/historical/fmsrfk.html")
        try:
            wait.until(EC.presence_of_element_located((By.ID, "label0")))
            Select(driver.find_element(By.ID, "label0")).select_by_value(str(year))

            stock_input = driver.find_element(By.ID, "label1")
            stock_input.clear()
            stock_input.send_keys(stock_code)

            search_btn = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "查詢")]')))
            driver.execute_script("arguments[0].click();", search_btn)

            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "tbody.is-last-page")))
            rows = driver.find_elements(By.CSS_SELECTOR, "tbody.is-last-page tr")
            for row in rows:
                cols = [cell.text.replace(",", "") for cell in row.find_elements(By.TAG_NAME, "td")]
                all_data.append(cols)

            print(f"✅ {stock_code} 民國 {year - 1911} 年完成")
            time.sleep(wait_time)

        except Exception as e:
            print(f"⚠️  {stock_code} 年 {year} 查詢失敗：{e}")
            continue

    return all_data


# 主程式開始
for idx, stock_id in enumerate(stock_ids):
    output_file = os.path.join(output_dir, f"{stock_id}.csv")

    if os.path.exists(output_file):
        print(f"⏭ 已存在 {stock_id}.csv，跳過")
        continue

    print(f"🚀 開始查詢 {stock_id}（第 {idx+1} / {len(stock_ids)}）")
    data = fetch_stock_data(stock_id)

    if data:
        df = pd.DataFrame(data, columns=["年", "月", "最高", "最低", "收盤", "成交量(千股)", "成交金額", "市值", "報酬率%"])
        df.to_csv(output_file, index=False, encoding="utf-8-sig")
        print(f"📁 已儲存：{output_file}")
    else:
        print(f"❌ 無資料：{stock_id}")

driver.quit()
print("🎉 全部查詢完畢")


⏭ 已存在 1101.csv，跳過
🚀 開始查詢 1102（第 2 / 1650）


KeyboardInterrupt: 

In [1]:
import os
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import UnexpectedAlertPresentException, NoAlertPresentException

# 設定
start_year = 2017
end_year = 2025
wait_range = (4.5, 7.5)  # 隨機等待秒數範圍
output_dir = "stock_data"
batch_size = 50  # 每批跑幾支股票
batch_delay = 300  # 每批間隔時間（秒）

# 建立輸出資料夾
os.makedirs(output_dir, exist_ok=True)

# 讀取股票代碼（去重排序）
with open("stock_ids.txt", "r", encoding="utf-8") as f:
    stock_ids = sorted(set(line.strip() for line in f if line.strip()))

# 初始化錯誤記錄
error_log = []

# 初始化瀏覽器
def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    return webdriver.Chrome(options=options)

driver = create_driver()
wait = WebDriverWait(driver, 15)

# 查詢資料函數（含 alert 處理）
def fetch_stock_data(stock_code):
    all_data = []

    for year in range(start_year, end_year + 1):
        try:
            driver.get("https://www.twse.com.tw/zh/trading/historical/fmsrfk.html")
            wait.until(EC.presence_of_element_located((By.ID, "label0")))

            Select(driver.find_element(By.ID, "label0")).select_by_value(str(year))

            stock_input = driver.find_element(By.ID, "label1")
            stock_input.clear()
            stock_input.send_keys(stock_code)

            search_btn = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "查詢")]')))
            driver.execute_script("arguments[0].click();", search_btn)

            # 偵測表格或 alert
            try:
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "tbody.is-last-page")))
                rows = driver.find_elements(By.CSS_SELECTOR, "tbody.is-last-page tr")
                for row in rows:
                    cols = [cell.text.replace(",", "") for cell in row.find_elements(By.TAG_NAME, "td")]
                    all_data.append(cols)

                print(f"✅ {stock_code} 民國 {year - 1911} 年完成")
            except:
                try:
                    alert = driver.switch_to.alert
                    print(f"⚠️  {stock_code} 民國 {year - 1911} 出現警告：{alert.text}")
                    alert.accept()
                    time.sleep(10)
                    return fetch_stock_data(stock_code)  # 重試整支股票
                except NoAlertPresentException:
                    print(f"❌ 未知錯誤：{stock_code} {year}")
        except Exception as e:
            print(f"❌ 查詢錯誤：{stock_code} {year}，{e}")
            continue

        time.sleep(random.uniform(*wait_range))  # 隨機等待

    return all_data

# 主流程：分批執行
for i in range(0, len(stock_ids), batch_size):
    batch = stock_ids[i:i + batch_size]
    print(f"🚀 開始第 {i // batch_size + 1} 批（共 {len(batch)} 支）")

    for stock_code in batch:
        output_file = os.path.join(output_dir, f"{stock_code}.csv")
        if os.path.exists(output_file):
            print(f"⏭ 已存在 {stock_code}.csv，跳過")
            continue

        data = fetch_stock_data(stock_code)
        if data:
            df = pd.DataFrame(data, columns=["年", "月", "最高", "最低", "收盤", "成交量(千股)", "成交金額", "市值", "報酬率%"])
            df.to_csv(output_file, index=False, encoding="utf-8-sig")
            print(f"📁 已儲存：{output_file}")
        else:
            error_log.append(stock_code)
            print(f"⚠️ 無資料或查詢失敗：{stock_code}")

    # 每批之間休息
    if i + batch_size < len(stock_ids):
        print(f"😴 等待 {batch_delay} 秒後跑下一批...")
        time.sleep(batch_delay)

# 關閉瀏覽器
driver.quit()

# 儲存錯誤 log
if error_log:
    with open("error_log.txt", "w", encoding="utf-8") as f:
        f.write("\n".join(error_log))
    print(f"⚠️ 已記錄錯誤股票至 error_log.txt，共 {len(error_log)} 筆")

print("🎉 批次查詢完成")


🚀 開始第 1 批（共 50 支）
⏭ 已存在 1101.csv，跳過
✅ 1102 民國 106 年完成
✅ 1102 民國 107 年完成


KeyboardInterrupt: 

In [18]:
import os
import requests
import pandas as pd
import time
import random

# 設定
START_YEAR = 2017
END_YEAR = 2025
MAX_RETRIES = 3
WAIT_RANGE = (1.0, 2.0)
STOCK_FILE = "stock_ids.txt"
OUTPUT_DIR = "output_data"

# 建立儲存資料夾
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 讀取股票代碼清單（string 處理）
with open(STOCK_FILE, "r", encoding="utf-8") as f:
    stock_list = sorted(set(line.strip() for line in f if line.strip()))

# 取得月報酬率資料 TWT54U
def fetch_twt54u(stock_id, year):
    url = "https://www.twse.com.tw/rwd/zh/fund/TWT54U"
    params = {
        "response": "json",
        "stockNo": stock_id,
        "year": year,
    }
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            resp = requests.get(url, params=params, timeout=10)
            if resp.ok:
                return resp.json().get("data", [])
        except Exception as e:
            print(f"⚠️ TWT54U 錯誤 [{stock_id} {year}] 第 {attempt} 次：{e}")
        time.sleep(2 * attempt)
    return []

# 取得財務指標 BWIBBU_d（PE, PBR, 殖利率）
def fetch_bwibbu(date_str):
    url = "https://www.twse.com.tw/exchangeReport/BWIBBU_d"
    params = {
        "response": "json",
        "date": date_str,
        "selectType": "ALL"
    }
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            resp = requests.get(url, params=params, timeout=10)
            if resp.ok:
                raw = resp.json().get("data", [])
                return {row[0].strip(): row[2:6] for row in raw}  # 股票代碼 : [收盤, PE, PBR, 殖利率]
        except Exception as e:
            print(f"⚠️ BWIBBU 錯誤 [{date_str}] 第 {attempt} 次：{e}")
        time.sleep(2 * attempt)
    return {}

# 主流程
for idx, stock_id in enumerate(stock_list):
    stock_id = stock_id.strip()
    output_path = os.path.join(OUTPUT_DIR, f"{stock_id}.csv")

    if os.path.exists(output_path):
        print(f"⏭ 已存在 {stock_id}.csv，跳過")
        continue

    print(f"\n🚀 開始查詢股票：{stock_id}（{idx+1}/{len(stock_list)}）")
    combined_rows = []

    for year in range(START_YEAR, END_YEAR + 1):
        twt_data = fetch_twt54u(stock_id, year)
        if not twt_data:
            print(f"⚠️ 無報酬率資料：{stock_id} 年 {year}")
            continue

        for row in twt_data:
            if len(row) < 10:
                print(f"⚠️ 略過格式錯誤 row：{row}")
                continue

            try:
                roc_year = int(row[0].strip())
                month = int(row[1].strip())
            except ValueError:
                print(f"⚠️ 資料欄位非數字，跳過 row：{row}")
                continue

            date_str = f"{roc_year + 1911}{month:02d}01"
            bwibbu_data = fetch_bwibbu(date_str)
            fin_data = bwibbu_data.get(stock_id, ["", "", "", ""])  # 收盤, PE, PBR, 殖利率

            combined_row = [stock_id, roc_year + 1911] + row + fin_data
            combined_rows.append(combined_row)
            print(f"✅ {stock_id} 民國{roc_year}年{month}月 OK")

            time.sleep(random.uniform(*WAIT_RANGE))

    # 儲存 CSV（若有資料）
    if combined_rows:
        columns = [
            "股票代碼", "西元年", "民國年", "月份", "最高", "最低", "收盤", "成交量(千股)",
            "成交金額", "市值", "報酬率%", "當月收盤", "本益比", "股價淨值比", "殖利率%"
        ]
        df = pd.DataFrame(combined_rows, columns=columns)
        df.to_csv(output_path, index=False, encoding="utf-8-sig")
        print(f"📁 資料已儲存至：{output_path}")
    else:
        print(f"❌ 無資料可寫入：{stock_id}")



🚀 開始查詢股票：1101（1/1650）
⚠️ 資料欄位非數字，跳過 row：['1102', '亞泥            ', '32,372,706', '32,250,253', '122,453', '0', '0', '0', '1,360,619', '1,039,240', '321,379', '9,294,504', '397,673', '1,013,001', '-615,328', '10,141,691', '231,859', '9,909,832', '9,738,336']
⚠️ 資料欄位非數字，跳過 row：['1101B', '台泥乙特        ', '0', '0', '0', '0', '0', '0', '0', '0', '0', '51,000', '71,000', '20,000', '51,000', '0', '0', '0', '51,000']
⚠️ 資料欄位非數字，跳過 row：['1109', '信大            ', '250,007', '309,994', '-59,987', '0', '0', '0', '0', '0', '0', '-2,229', '50,000', '52,229', '-2,229', '0', '0', '0', '-62,216']
⚠️ 資料欄位非數字，跳過 row：['1103', '嘉泥            ', '964,010', '1,231,820', '-267,810', '0', '0', '0', '0', '0', '0', '-8,724', '65,000', '73,724', '-8,724', '0', '0', '0', '-276,534']
⚠️ 資料欄位非數字，跳過 row：['1110', '東泥            ', '577,000', '910,000', '-333,000', '0', '0', '0', '0', '0', '0', '-4,000', '68,000', '72,000', '-4,000', '0', '0', '0', '-337,000']
⚠️ 資料欄位非數字，跳過 row：['1108', '幸福            ', '483,165', '87

KeyboardInterrupt: 

In [16]:
import requests
import pandas as pd
import time
import random

# 設定
START_YEAR = 2017
END_YEAR = 2025
MAX_RETRIES = 3
WAIT_RANGE = (1.0, 2.0)  # 每筆延遲時間
STOCK_FILE = "stock_ids.txt"
OUTPUT_FILE = "twse_all_data.csv"

# 讀入股票清單
with open(STOCK_FILE, "r", encoding="utf-8") as f:
    stock_list = sorted(set(line.strip() for line in f if line.strip()))

# 儲存所有資料
all_rows = []

# 抓取函數
def fetch_stock_data(stock_id, year):
    url = "https://www.twse.com.tw/rwd/zh/fund/TWT54U"
    params = {
        "response": "json",
        "stockNo": stock_id,
        "year": year,
    }

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            resp = requests.get(url, params=params, timeout=10)
            if resp.ok:
                data = resp.json()
                return data.get("data", [])
            else:
                print(f"❌ HTTP錯誤 [{stock_id} 年 {year}]：{resp.status_code}")
        except Exception as e:
            print(f"⚠️  發生錯誤 [{stock_id} 年 {year}] 嘗試第 {attempt} 次：{e}")
        time.sleep(2 * attempt)
    return []

# 主程式
for idx, stock_id in enumerate(stock_list):
    print(f"🚀 處理第 {idx+1}/{len(stock_list)}：{stock_id}")

    for year in range(START_YEAR, END_YEAR + 1):
        data = fetch_stock_data(stock_id, year)
        if data:
            for row in data:
                all_rows.append([stock_id, year] + row)
            print(f"✅  完成 {stock_id} 年 {year}")
        else:
            print(f"⚠️  無資料或失敗：{stock_id} 年 {year}")
        time.sleep(random.uniform(*WAIT_RANGE))

# 儲存 CSV
columns = ["股票代碼", "西元年", "民國年", "月份", "最高", "最低", "收盤", "成交量(千股)", "成交金額", "市值", "報酬率%"]
df = pd.DataFrame(all_rows, columns=columns)
df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")
print(f"\n📁 完成所有資料查詢，共 {len(df)} 筆，已儲存為：{OUTPUT_FILE}")


🚀 處理第 1/48：1101
✅  完成 1101 年 2017
✅  完成 1101 年 2018
✅  完成 1101 年 2019
✅  完成 1101 年 2020
✅  完成 1101 年 2021
✅  完成 1101 年 2022
✅  完成 1101 年 2023
✅  完成 1101 年 2024
✅  完成 1101 年 2025
🚀 處理第 2/48：1102
✅  完成 1102 年 2017
✅  完成 1102 年 2018
✅  完成 1102 年 2019
✅  完成 1102 年 2020
✅  完成 1102 年 2021
✅  完成 1102 年 2022
✅  完成 1102 年 2023


KeyboardInterrupt: 

In [7]:
import requests
import pandas as pd
import time
import random

# 設定
START_YEAR = 2017
END_YEAR = 2025
MAX_RETRIES = 3
WAIT_RANGE = (1.0, 2.0)
STOCK_FILE = "stock_ids.txt"
OUTPUT_FILE = "twse_all_data.csv"

# 讀股票清單
with open(STOCK_FILE, "r", encoding="utf-8") as f:
    stock_list = sorted(set(line.strip() for line in f if line.strip()))

# 儲存所有資料
all_rows = []

# 抓 TWT54U 報酬率資料
def fetch_stock_data(stock_id, year):
    url = "https://www.twse.com.tw/rwd/zh/fund/TWT54U"
    params = {
        "response": "json",
        "stockNo": stock_id,
        "year": year,
    }
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            resp = requests.get(url, params=params, timeout=10)
            if resp.ok:
                return resp.json().get("data", [])
        except Exception as e:
            print(f"⚠️ TWT54U 錯誤 [{stock_id} {year}] 嘗試 {attempt}：{e}")
        time.sleep(2 * attempt)
    return []

# 抓 BWIBBU_d 每月基本面（PE, PBR）
def fetch_bwibbu(date_str):
    url = "https://www.twse.com.tw/exchangeReport/BWIBBU_d"
    params = {
        "response": "json",
        "date": date_str,
        "selectType": "ALL"
    }
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            resp = requests.get(url, params=params, timeout=10)
            if resp.ok:
                data = resp.json().get("data", [])
                return {row[0].strip(): row[3:6] for row in data}  # PE, PBR, 殖利率
        except Exception as e:
            print(f"⚠️ BWIBBU_d 錯誤 [{date_str}] 嘗試 {attempt}：{e}")
        time.sleep(2 * attempt)
    return {}

# 主程式
for idx, stock_id in enumerate(stock_list):
    print(f"\n🚀 處理第 {idx+1}/{len(stock_list)} 檔：{stock_id}")

    for year in range(START_YEAR, END_YEAR + 1):
        data = fetch_stock_data(stock_id, year)
        if data:
            for row in data:
                try:
                    roc_year = int(row[0].strip())
                    month = int(row[1].strip())
                except (ValueError, IndexError):
                    print(f"⚠️ 無效資料，跳過 row：{row}")
                    continue

                # 每月查 BWIBBU 的基本面資料
                date_str = f"{roc_year + 1911}{month:02d}01"
                bwibbu = fetch_bwibbu(date_str)
                pe, pbr, _ = bwibbu.get(stock_id, ["", "", ""])  # 只取 PE/PBR（殖利率略過）

                all_rows.append([stock_id, year] + row + [pe, pbr, ""])  # ROE 佔空位
            print(f"✅ 完成 {stock_id} 年 {year}")
        else:
            print(f"⚠️ 無資料或失敗：{stock_id} 年 {year}")
        time.sleep(random.uniform(*WAIT_RANGE))

# 儲存 CSV
columns = [
    "股票代碼", "西元年", "民國年", "月份", "最高", "最低", "收盤", "成交量(千股)",
    "成交金額", "市值", "報酬率%", "本益比", "股價淨值比", "股東權益報酬率"
]
df = pd.DataFrame(all_rows, columns=columns)
df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")
print(f"\n📁 全部完成，共 {len(df)} 筆資料，已儲存至：{OUTPUT_FILE}")



🚀 處理第 1/1650 檔：1101
⚠️ 無效資料，跳過 row：['1102', '亞泥            ', '32,372,706', '32,250,253', '122,453', '0', '0', '0', '1,360,619', '1,039,240', '321,379', '9,294,504', '397,673', '1,013,001', '-615,328', '10,141,691', '231,859', '9,909,832', '9,738,336']
⚠️ 無效資料，跳過 row：['1101B', '台泥乙特        ', '0', '0', '0', '0', '0', '0', '0', '0', '0', '51,000', '71,000', '20,000', '51,000', '0', '0', '0', '51,000']
⚠️ 無效資料，跳過 row：['1109', '信大            ', '250,007', '309,994', '-59,987', '0', '0', '0', '0', '0', '0', '-2,229', '50,000', '52,229', '-2,229', '0', '0', '0', '-62,216']
⚠️ 無效資料，跳過 row：['1103', '嘉泥            ', '964,010', '1,231,820', '-267,810', '0', '0', '0', '0', '0', '0', '-8,724', '65,000', '73,724', '-8,724', '0', '0', '0', '-276,534']
⚠️ 無效資料，跳過 row：['1110', '東泥            ', '577,000', '910,000', '-333,000', '0', '0', '0', '0', '0', '0', '-4,000', '68,000', '72,000', '-4,000', '0', '0', '0', '-337,000']
⚠️ 無效資料，跳過 row：['1108', '幸福            ', '483,165', '874,000', '-390,835', 

KeyboardInterrupt: 

In [20]:
import requests
import pandas as pd
import time
import random

# === 設定區 ===
START_YEAR = 2017
END_YEAR = 2025
MAX_RETRIES = 3
WAIT_RANGE = (2.0, 4.0)
STOCK_FILE = "stock_ids.txt"
OUTPUT_FILE = "twse_all_data_fast.csv"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/120.0.0.0 Safari/537.36"
}

# === Step 1：讀入股票代碼 ===
with open("stock_ids.txt", "r", encoding="utf-8") as f:
    stock_list = sorted(set(line.strip() for line in f if line.strip()))

# === Step 2：快取 BWIBBU 所有年月資料 ===
bwibbu_cache = {}

def load_bwibbu_for_months(start_year, end_year):
    print("📦 開始預抓 BWIBBU 財報資料...")
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            date_str = f"{year}{month:02d}01"
            url = "https://www.twse.com.tw/exchangeReport/BWIBBU_d"
            params = {
                "response": "json",
                "date": date_str,
                "selectType": "ALL"
            }
            for attempt in range(MAX_RETRIES):
                try:
                    resp = requests.get(url, headers=HEADERS, params=params, timeout=10)
                    if resp.ok:
                        raw = resp.json().get("data", [])
                        bwibbu_cache[date_str] = {
                            row[0].strip(): row[3:6] for row in raw  # PE, PBR, Yield
                        }
                        print(f"✅ 已快取：{date_str}")
                        break
                except Exception as e:
                    print(f"⚠️ BWIBBU 下載失敗 [{date_str}]：{e}")
                    time.sleep(5 + attempt * 3)
            time.sleep(random.uniform(0.8, 1.5))

# === Step 3：查詢報酬率 API ===
def fetch_twt54u(stock_id, year):
    url = "https://www.twse.com.tw/rwd/zh/fund/TWT54U"
    params = {
        "response": "json",
        "stockNo": stock_id,
        "year": year,
    }
    for attempt in range(MAX_RETRIES):
        try:
            resp = requests.get(url, headers=HEADERS, params=params, timeout=10)
            if resp.ok:
                return resp.json().get("data", [])
        except Exception as e:
            print(f"⚠️ TWT54U 錯誤 [{stock_id} {year}]：{e}")
            time.sleep(5 + attempt * 3)
    return []

# === Step 4：主流程 ===
all_rows = []

# 📥 先載入 BWIBBU 快取
load_bwibbu_for_months(START_YEAR, END_YEAR)

# 🔁 開始查詢每支股票
for idx, stock_id in enumerate(stock_list):
    print(f"\n🚀 處理第 {idx+1}/{len(stock_list)}：{stock_id}")

    for year in range(START_YEAR, END_YEAR + 1):
        twt_data = fetch_twt54u(stock_id, year)
        if not twt_data:
            print(f"⚠️ 無報酬率資料：{stock_id} 年 {year}")
            continue

        for row in twt_data:
            try:
                roc_year = int(row[0].strip())
                month = int(row[1].strip())
            except (ValueError, IndexError):
                print(f"⚠️ 資料格式錯誤，跳過 row：{row}")
                continue

            date_str = f"{roc_year + 1911}{month:02d}01"
            pe, pbr, _ = bwibbu_cache.get(date_str, {}).get(stock_id, ["", "", ""])

            all_rows.append([stock_id, roc_year + 1911] + row + [pe, pbr])
        print(f"✅ 完成 {stock_id} 年 {year}")

        time.sleep(random.uniform(*WAIT_RANGE))

# === Step 5：輸出結果 ===
columns = [
    "股票代碼", "西元年", "民國年", "月份", "最高", "最低", "收盤", "成交量(千股)",
    "成交金額", "市值", "報酬率%", "本益比", "股價淨值比"
]
df = pd.DataFrame(all_rows, columns=columns)
df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")
print(f"\n🎉 全部完成！共 {len(df)} 筆，已儲存至：{OUTPUT_FILE}")


📦 開始預抓 BWIBBU 財報資料...
⚠️ BWIBBU 下載失敗 [20170101]：('Connection aborted.', ConnectionResetError(10054, '遠端主機已強制關閉一個現存的連線。', None, 10054, None))


KeyboardInterrupt: 

In [24]:
pip install tejapi

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
import tejapi
import pandas as pd

# Set your TEJ API key
tejapi.ApiConfig.api_key = '0JKBCbim7gp6QaRpw3OuH5unatZ096'
info = tejapi.ApiConfig.info()


# 載入股票代碼列表
stock_ids = []
with open('stock_ids.txt', 'r') as file:
    stock_ids = [line.strip() for line in file.readlines()]

# 定義一個函式來抓取每個股票的資料
def fetch_stock_data(stock_ids):
    stock_data = []

    for stock_id in stock_ids:
        try:
            # 查詢股票的月平均收盤價
            price_data = tejapi.get('TWN/APRCD', coid=stock_id, opts={'columns': ['mdate', 'close_d']})
            
            # 查詢股票的本益比和股價淨值比
            pe_pb_data = tejapi.get('TWN/PEPB', coid=stock_id, opts={'columns': ['mdate', 'pe_ratio', 'pb_ratio']})

            # 整合資料
            combined_data = {
                'stock_id': stock_id,
                'price_data': price_data,
                'pe_pb_data': pe_pb_data
            }
            stock_data.append(combined_data)
        except Exception as e:
            print(f"無法抓取股票 {stock_id} 的資料: {e}")

    return stock_data

# 抓取資料
stock_data = fetch_stock_data(stock_ids)

# 顯示第一檔股票的資料作為範例
print(stock_data[0] if stock_data else "沒有抓取到資料")


無法抓取股票 1101 的資料: (Status 400) (Tej Error PDB003) 您沒有存取資料表的權限。
無法抓取股票 1102 的資料: (Status 400) (Tej Error PDB003) 您沒有存取資料表的權限。
無法抓取股票 1103 的資料: (Status 400) (Tej Error PDB003) 您沒有存取資料表的權限。
無法抓取股票 1104 的資料: (Status 400) (Tej Error PDB003) 您沒有存取資料表的權限。
無法抓取股票 1108 的資料: (Status 400) (Tej Error PDB003) 您沒有存取資料表的權限。
無法抓取股票 1109 的資料: (Status 400) (Tej Error PDB003) 您沒有存取資料表的權限。
無法抓取股票 1110 的資料: (Status 400) (Tej Error PDB003) 您沒有存取資料表的權限。
無法抓取股票 1201 的資料: (Status 400) (Tej Error PDB003) 您沒有存取資料表的權限。
無法抓取股票 1203 的資料: (Status 400) (Tej Error PDB003) 您沒有存取資料表的權限。
無法抓取股票 1210 的資料: (Status 400) (Tej Error PDB003) 您沒有存取資料表的權限。
無法抓取股票 1213 的資料: (Status 400) (Tej Error PDB003) 您沒有存取資料表的權限。


KeyboardInterrupt: 

In [4]:
import requests
import pandas as pd

# 設定 API URL
stock_day_avg_url = "https://openapi.twse.com.tw/exchangeReport/STOCK_DAY_AVG_ALL"
pe_pb_url = "https://openapi.twse.com.tw/exchangeReport/BWIBBU_ALL"
financial_report_url = "https://openapi.twse.com.tw/opendata/t187ap06_L_ci"

# 設定股票代碼列表（從 txt 檔案中讀取）
stock_ids = []
with open('stock_ids.txt', 'r') as file:
    stock_ids = [line.strip() for line in file.readlines()]

# 獲取日收盤價與月平均價
def fetch_stock_day_avg(stock_id):
    params = {'stockNo': stock_id}
    response = requests.get(stock_day_avg_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"無法取得股票 {stock_id} 的日收盤與月平均價資料")
        return None

# 獲取本益比、股價淨值比
def fetch_pe_pb(stock_id):
    params = {'stockNo': stock_id}
    response = requests.get(pe_pb_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"無法取得股票 {stock_id} 的本益比與股價淨值比資料")
        return None

# 獲取綜合損益表（財報資料）
def fetch_financial_report(stock_id):
    params = {'stockNo': stock_id}
    response = requests.get(financial_report_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"無法取得股票 {stock_id} 的財報資料")
        return None

# 保存資料到 CSV 檔案
def save_to_csv(stock_id, stock_data, filename):
    df = pd.DataFrame(stock_data)
    df.to_csv(f'{filename}_{stock_id}.csv', index=False)
    print(f"已儲存 {stock_id} 的資料至 {filename}_{stock_id}.csv")

# 主程式
def main():
    for stock_id in stock_ids:
        # 爬取每日收盤價與月平均價
        stock_day_avg_data = fetch_stock_day_avg(stock_id)
        if stock_day_avg_data:
            save_to_csv(stock_id, stock_day_avg_data, 'stock_day_avg')
        
        # 爬取本益比與股價淨值比
        pe_pb_data = fetch_pe_pb(stock_id)
        if pe_pb_data:
            save_to_csv(stock_id, pe_pb_data, 'pe_pb')
        
        # 爬取財報資料
        financial_report_data = fetch_financial_report(stock_id)
        if financial_report_data:
            save_to_csv(stock_id, financial_report_data, 'financial_report')

# 執行主程式
if __name__ == "__main__":
    main()


無法取得股票 1101 的日收盤與月平均價資料
無法取得股票 1101 的本益比與股價淨值比資料
無法取得股票 1101 的財報資料
無法取得股票 1102 的日收盤與月平均價資料
無法取得股票 1102 的本益比與股價淨值比資料
無法取得股票 1102 的財報資料
無法取得股票 1103 的日收盤與月平均價資料
無法取得股票 1103 的本益比與股價淨值比資料
無法取得股票 1103 的財報資料
無法取得股票 1104 的日收盤與月平均價資料
無法取得股票 1104 的本益比與股價淨值比資料
無法取得股票 1104 的財報資料
無法取得股票 1108 的日收盤與月平均價資料
無法取得股票 1108 的本益比與股價淨值比資料
無法取得股票 1108 的財報資料
無法取得股票 1109 的日收盤與月平均價資料
無法取得股票 1109 的本益比與股價淨值比資料
無法取得股票 1109 的財報資料
無法取得股票 1110 的日收盤與月平均價資料
無法取得股票 1110 的本益比與股價淨值比資料
無法取得股票 1110 的財報資料
無法取得股票 1201 的日收盤與月平均價資料
無法取得股票 1201 的本益比與股價淨值比資料
無法取得股票 1201 的財報資料
無法取得股票 1203 的日收盤與月平均價資料
無法取得股票 1203 的本益比與股價淨值比資料
無法取得股票 1203 的財報資料
無法取得股票 1210 的日收盤與月平均價資料
無法取得股票 1210 的本益比與股價淨值比資料
無法取得股票 1210 的財報資料
無法取得股票 1213 的日收盤與月平均價資料
無法取得股票 1213 的本益比與股價淨值比資料
無法取得股票 1213 的財報資料
無法取得股票 1215 的日收盤與月平均價資料
無法取得股票 1215 的本益比與股價淨值比資料
無法取得股票 1215 的財報資料
無法取得股票 1216 的日收盤與月平均價資料
無法取得股票 1216 的本益比與股價淨值比資料
無法取得股票 1216 的財報資料
無法取得股票 1217 的日收盤與月平均價資料
無法取得股票 1217 的本益比與股價淨值比資料
無法取得股票 1217 的財報資料
無法取得股票 1218 的日收盤與月平均價資料
無法取得股票 1218 的本益比與股價淨值比資料
無法取得股票 1218 的

KeyboardInterrupt: 

In [16]:
import pandas as pd
import os

# 設定包含所有 Excel 檔案的資料夾路徑
folder_path = '手按壓縮檔季月底均'

# 建立一個空的 DataFrame 用來合併所有資料
all_data = pd.DataFrame()

# 遍歷資料夾中的所有 Excel 檔案
for filename in os.listdir(folder_path):
    if filename.endswith(".xls") or filename.endswith(".xlsx"):
        file_path = os.path.join(folder_path, filename)

        # 讀取第七頁資料
        df = pd.read_excel(file_path, sheet_name=8)  # 第八頁的索引是 6

        # 取得檔案的月份，假設檔案名稱中包含月份資訊（例如：202503、201703）
        file_month = filename.split('.')[0][-6:]  # 假設檔名格式是 YYYYMM 或其他類似格式
        
        # 新增一欄「月份」並填入檔案月份
        df['月份'] = file_month
        
        # 合併資料到 all_data
        all_data = pd.concat([all_data, df], ignore_index=True)

# 儲存所有合併後的資料到 CSV 或 Excel
output_file_path = os.path.join(folder_path, 'merged_data.csv')
all_data.to_csv(output_file_path, index=False)
print(f"資料已儲存至: {output_file_path}")



資料已儲存至: 手按壓縮檔季月底均\merged_data.csv


In [6]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
